# Libs and Inputs

In [7]:
from __future__ import division
from pyomo.environ import *
import numpy as np
import time
from tqdm import tqdm
from GetIdxInOutRadious import GetIdxOutRadious, GetIdxInRadious_V1
import matlab.engine
from multiprocessing import Pool
import multiprocessing
from tqdm import tqdm
from joblib import Parallel, delayed
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt


LCOE_Max=range(200,30,-2) #LCOEs investigated
Radious=30 #Radious for the energy collection system

ShapeFileCoast="./GEO_data/ne_10m_coastline.shp"
ShapeFileStates="./GEO_data/ne_10m_admin_1_states_provinces_lines.shp"

WindKiteTrsData=np.load('PreprocessedData.npz',allow_pickle=True)

ResultsFileName='PortfolioOptimizationWindKite'

# Tasks:
#1. Create a limit in the number of turbines for each tech based on the max transmission (eg- wind 100, kite- 50)
#2. Simulation 1- Optimize with only wind for a maximum num of turbines

#3. Simulation 2- Optimize with wind and kite same transmission system
#4. Simulation 3- Optimize with wind and kite kite with its own transmission system connecting to the offshore wind platform


# Stage 1 Optimization

In [2]:
#LF kite and optimal kite design at each site location

## Prepare Data For the Optimization Model

In [8]:
#Wind Data
WindEnergy=WindKiteTrsData['WindEnergy']
WindLatLong=WindKiteTrsData['WindLatLong']
AnnualizedCostWind=WindKiteTrsData['AnnualizedCostWind']
MaxNumWindPerSite=WindKiteTrsData['MaxNumWindPerSite']

#Kite Data
KiteEnergy=WindKiteTrsData['KiteEnergy']
KiteLatLong=WindKiteTrsData['KiteLatLong']
AnnualizedCostKite=WindKiteTrsData['AnnualizedCostKite']
MaxNumKitesPerSite=WindKiteTrsData['MaxNumKitesPerSite']

#Transmission Data
AnnualizedCostTransmission=WindKiteTrsData['Wind_AnnualizedCostTransmission']
TransLatLong=WindKiteTrsData['Wind_TransLatLong']
EfficiencyTransmission=WindKiteTrsData['Wind_EfficiencyTransmission']
MaxPowerTransmission=float(WindKiteTrsData['Wind_MaxPowerTransmission'])

TimeStepHours=WindKiteTrsData['TimeStepHours'] #Number of hours for each time step

In [9]:
#Vectorize maximum number of turbines per site location per technology
Nu=np.concatenate((MaxNumWindPerSite,MaxNumKitesPerSite))

#Vectorize annualized cost for each site location and per technology
AnnCost=np.concatenate((AnnualizedCostWind,AnnualizedCostKite)) #Annualized cost [$/Year]

#Vectorize energy generation in each site location and energy resource
EnergyGeneration=np.concatenate((WindEnergy,KiteEnergy),axis=0) #MW ()

NumWindSites=WindEnergy.shape[0]
NumKiteSites=KiteEnergy.shape[0]

NumTrasmissionSites=TransLatLong.shape[0]

NumTimeSteps=WindEnergy.shape[1]

## Build Optimization Model Structure

In [17]:
MILP = ConcreteModel()
BigM=10000
# Create Sets
MILP.SiteWind = RangeSet(0,NumWindSites-1)
MILP.SiteKite = RangeSet(0,NumKiteSites-1)
MILP.SiteTrs  = RangeSet(0,NumTrasmissionSites-1)

MILP.TimeSteps = RangeSet(0,NumTimeSteps-1)

# Create Variables
MILP.Y_Wind = Var(MILP.SiteWind, domain=NonNegativeIntegers)# Integer variable to track the number of wind turbines used per site location
MILP.Y_Kite = Var(MILP.SiteKite, domain=NonNegativeIntegers)# Integer variable to track the number of wind turbines used per site location

MILP.s = Var(MILP.SiteTrs, domain=Binary)# Binary variable to track the center of the energy collection system
MILP.Delta = Var(MILP.TimeSteps, domain=NonNegativeReals) #Curtailment variable

#Objective Function
def objective_rule(MILP):   
    EGWind=sum(MILP.Y_Wind[i]*np.average(WindEnergy[i,:]) for i in MILP.SiteWind) #Energy generation from wind turbines
    EGKite=sum(MILP.Y_Kite[i]*np.average(KiteEnergy[i,:]) for i in MILP.SiteKite) #Energy generation from kite turbines

    TotalCurtailment=sum(MILP.Delta[t] for t in MILP.TimeSteps)/NumTimeSteps #Average curtailment MW

    Obj=(EGWind+EGKite-TotalCurtailment)#*24*365#Mwh/year
    return Obj

MILP.OBJ = Objective(rule = objective_rule, sense=maximize)

#Constraints

#Maximum number of turbines per site location wind
def MaxTurbinesCell_Wind_rule(MILP,i):
    return MILP.Y_Wind[i]<=MaxNumWindPerSite[i]
MILP.Turbines_Cell_Wind = Constraint(MILP.SiteWind, rule=MaxTurbinesCell_Wind_rule)

#Maximum number of turbines per site location kite
def MaxTurbinesCell_Kite_rule(MILP,i):
    return MILP.Y_Kite[i]<=MaxNumKitesPerSite[i]
MILP.Turbines_Cell_Kite = Constraint(MILP.SiteKite, rule=MaxTurbinesCell_Kite_rule)


#Curtailment constraint
def Curtailment_rule(MILP,t):
    EGWind=sum(MILP.Y_Wind[i]*WindEnergy[i,t] for i in MILP.SiteWind) #Energy generation from wind turbines
    EGKite=sum(MILP.Y_Kite[i]*KiteEnergy[i,t] for i in MILP.SiteKite) #Energy generation from kite turbines

    return -MILP.Delta[t]+ EGWind+ EGKite<=MaxPowerTransmission

MILP.Curtailment = Constraint(MILP.TimeSteps, rule=Curtailment_rule)


#---Choose center collection system - Start
MILP.ChooseOneCircle= Constraint(expr=sum(MILP.s[i] for i in MILP.SiteTrs)==1)

#Get the sites that are out of the radious of the center of the collection system
IdxOutWind=GetIdxOutRadious(TransLatLong, WindLatLong,Radious)
IdxOutKite=GetIdxOutRadious(TransLatLong, KiteLatLong,Radious)

def MaximumRadious(MILP,i):  
    SumWind_s=sum(MILP.Y_Wind[j] for j in IdxOutWind[i])
    SumKite_s=sum(MILP.Y_Kite[j] for j in IdxOutKite[i])

    return SumWind_s+SumKite_s<=(1-MILP.s[i])*BigM # 300 is a big M for the maximum total number of turbines installed       

MILP.Maximum_Radious = Constraint(MILP.SiteTrs, rule=MaximumRadious)
#---Choose center collection system - End


#LCOE Target
def LCOETarget_S1(MILP,LCOE_Max):  
    EG_Wind=sum(MILP.Y_Wind[i]*np.average(WindEnergy[i,:]) for i in MILP.SiteWind) #Average MW for wind
    EG_Kite=sum(MILP.Y_Kite[i]*np.average(KiteEnergy[i,:]) for i in MILP.SiteKite) #Average MW for kite
    TotalCurtailment=sum(MILP.Delta[i] for i in MILP.TimeSteps)/NumTimeSteps #Average curtailment MW
    
    MWh=(EG_Kite+EG_Wind-TotalCurtailment)*24*365 #MWh

    Cost_Wind=sum(MILP.Y_Wind[i]*AnnualizedCostWind[i] for i in MILP.SiteWind)
    Cost_Kite=sum(MILP.Y_Kite[i]*AnnualizedCostKite[i] for i in MILP.SiteKite)
    Cost_Transmission=sum(MILP.s[i]*AnnualizedCostTransmission[i] for i in MILP.SiteTrs)
    Cost=Cost_Wind+Cost_Kite+Cost_Transmission

    return Cost<=LCOE_Max*MWh  

#Limit the number of turbines
#MILP.SetNumTurbines= Constraint(expr=sum(MILP.Y_Kite[i] for i in MILP.SiteKite)==315) #optimal solution with only wind turbines
#MILP.SetNumTurbines= Constraint(expr=sum(MILP.Y_Kite[i] for i in MILP.SiteKite)==122) #optimal solution with only wind turbines
#MILP.SetNumTurbines= Constraint(expr=sum(MILP.Y_Wind[i] for i in MILP.SiteWind)==122)
MILP.SetNumTurbines= Constraint(expr=sum(MILP.Y_Wind[i]*np.average(WindEnergy[i,:]) for i in MILP.SiteWind)>=315)

opt = SolverFactory('gurobi', solver_io="python")
opt.options['mipgap'] = 0.05
#opt.options['max_iter'] = 500

## Solve Portfolio Optimization Stage 1 

In [18]:
SaveFeasibility, Save_LCOETarget, Save_LCOE_Achieved, SaveTotalMWAvg = list(), list(), list(), list()
SaveYWind, SaveYKite, SaveYTrans, SaveCurtail = list(), list(), list(), list()

LowestLCOE=10**10
for LCOE_Idx in tqdm(range(len(LCOE_Max))):
    LCOETarget=LCOE_Max[LCOE_Idx]
    
    if LCOETarget<LowestLCOE:    
        Bypass=0
        #Upperbound For the LCOE Activate Constraint
        LCOE_Target=LCOETarget_S1(MILP,LCOETarget)
        MILP.LCOE_Target = Constraint(rule=LCOE_Target)
        print("Running Model With LCOE= %.2f" % LCOETarget)
        
        try:
            results=opt.solve(MILP, tee=False)
        except:
            Bypass=1
            MILP.del_component(MILP.LCOE_Target)  
    
        if Bypass==0:
            if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
                SaveFeasibility.append(1)
                Save_LCOETarget.append(LCOETarget)
                
                Optimal_Y_Kite =np.array([MILP.Y_Kite.get_values()[j] for j in MILP.SiteKite])
                Optimal_Y_Wind =np.array([MILP.Y_Wind.get_values()[j] for j in MILP.SiteWind])
                Optimal_Y_Trans=np.array([MILP.s.get_values()[j] for j in MILP.SiteTrs])
                Curtailment=np.array([MILP.Delta.get_values()[j] for j in MILP.TimeSteps])

                SaveYWind.append(Optimal_Y_Wind)
                SaveYKite.append(Optimal_Y_Kite)
                SaveYTrans.append(Optimal_Y_Trans)
                SaveCurtail.append(Curtailment)

                #Current LCOE
                EG_Wind=np.sum(Optimal_Y_Wind*np.average(WindEnergy,axis=1)) #MW avg
                EG_Kite=np.sum(Optimal_Y_Kite*np.average(KiteEnergy,axis=1)) #MW avg
                TotalCurtailment=np.sum(Curtailment)/NumTimeSteps #MW avg
                MWh=(EG_Kite+EG_Wind-TotalCurtailment)*24*365 #MWh

                Cost_Wind  = sum(Optimal_Y_Wind*AnnualizedCostWind)
                Cost_Kite  = sum(Optimal_Y_Kite*AnnualizedCostKite)
                Cost_Trans = sum(Optimal_Y_Trans*AnnualizedCostTransmission)

                Cost=Cost_Wind+Cost_Kite+Cost_Trans
                CurrentLCOE=Cost/MWh
                
                Save_LCOE_Achieved.append(CurrentLCOE)
                SaveTotalMWAvg.append(value(MILP.OBJ))
                LowestLCOE=CurrentLCOE
                
                print("MW Wind: %.2f,\nMW Kite: %.2f,\nMW Curtailment: %.2f,\nMW Total: %.2f\n" % (EG_Wind,EG_Kite,TotalCurtailment,EG_Wind+EG_Kite-TotalCurtailment))

                
                #Delete constraint for its modification in the next step of the for loop
                MILP.del_component(MILP.LCOE_Target)
            
            else:# Something else is wrong
                MILP.del_component(MILP.LCOE_Target)
                SaveFeasibility.append(0)
                Save_LCOETarget.append(None)
                Save_LCOE_Achieved.append(None)
                SaveYWind.append(None)
                SaveYKite.append(None)
                SaveYTrans.append(None)
                SaveCurtail.append(None)    
                SaveTotalMWAvg.append(None)     

                break


  0%|          | 0/85 [00:00<?, ?it/s]

Running Model With LCOE= 200.00


  1%|          | 1/85 [00:09<13:00,  9.29s/it]

MW Wind: 618.17,
MW Kite: 903.01,
MW Curtailment: 926.39,
MW Total: 594.80

Running Model With LCOE= 198.00


  2%|▏         | 2/85 [00:16<11:01,  7.97s/it]

...

 61%|██████    | 52/85 [03:06<03:20,  6.08s/it]

MW Wind: 316.81,
MW Kite: 326.30,
MW Curtailment: 132.07,
MW Total: 511.05

Running Model With LCOE= 96.00


 62%|██████▏   | 53/85 [03:12<03:16,  6.14s/it]

MW Wind: 316.81,
MW Kite: 196.61,
MW Curtailment: 64.08,
MW Total: 449.34

Running Model With LCOE= 94.00
    model.name="unknown";
      - termination condition: infeasibleOrUnbounded
      - message from solver: <undefined>


 62%|██████▏   | 53/85 [03:18<01:59,  3.74s/it]


In [19]:
#Save Results
np.savez("./ResultsPortOPT/"+ResultsFileName +"Stage1LF"+ ".npz", 
        #Wind Data
        WindEnergy=WindEnergy,
        WindLatLong=WindLatLong,
        AnnualizedCostWind=AnnualizedCostWind,
        MaxNumWindPerSite=MaxNumWindPerSite,
        #Kite Data
        KiteEnergy=KiteEnergy,
        KiteLatLong=KiteLatLong,
        AnnualizedCostKite=AnnualizedCostKite,
        MaxNumKitesPerSite=MaxNumKitesPerSite,
        #Transmission Data
        AnnualizedCostTransmission=AnnualizedCostTransmission,
        TransLatLong=TransLatLong,
        EfficiencyTransmission=EfficiencyTransmission,
        MaxPowerTransmission=MaxPowerTransmission,
        TimeStepHours=TimeStepHours,
        #Solutions
        SaveFeasibility=SaveFeasibility,
        Save_LCOETarget=Save_LCOETarget,
        Save_LCOE_Achieved=Save_LCOE_Achieved,
        SaveYWind=SaveYWind,
        SaveYKite=SaveYKite,
        SaveYTrans=SaveYTrans,
        SaveCurtail=SaveCurtail,
        SaveTotalMWAvg=SaveTotalMWAvg)



c:\Users\Remote\anaconda3\envs\Gurobi\lib\site-packages\numpy\lib\npyio.py:696: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


# Stage 2 Optimization

## Convert Stage 1 Solution to Stage 2 Input

In [21]:
#Data for Stage 2
CaseInvestigated=-2
MaxNumDesigns=1
CaseKite_Idx=np.where(SaveYKite[CaseInvestigated])[0]
CaseTrans_Idx=np.where(SaveYTrans[CaseInvestigated])[0][0]

IdxInWind=GetIdxInRadious_V1(TransLatLong[CaseTrans_Idx,:], WindLatLong, Radious) #Kites inside radious of energy collection system
IdxInKite=GetIdxInRadious_V1(TransLatLong[CaseTrans_Idx,:], KiteLatLong, Radious) #Kites inside radious of energy collection system

#transmission location
TransLoc=np.where(SaveYTrans[CaseInvestigated])[0][0]

#update wind data
NumWindSites_s2=len(IdxInWind)
WindEnergy_s2=WindEnergy[IdxInWind,:]
WindLatLong_s2=WindLatLong[IdxInWind,:]
AnnualizedCostWind_s2=AnnualizedCostWind[IdxInWind]
MaxNumWindPerSite_s2=MaxNumWindPerSite[IdxInWind]

#update kite data
NumKiteSites_s2=len(IdxInKite)
NumKiteDesigns_s2=len(IdxInKite)
MaxNumKitesPerSite_s2=MaxNumKitesPerSite[IdxInKite]
KiteLatLong_s2=KiteLatLong[IdxInKite,:]
AnnualizedCostKite_s2=AnnualizedCostKite[IdxInKite]

KiteEnergy_s2=np.zeros((NumKiteSites_s2,NumKiteDesigns_s2,NumTimeSteps))
LCOETarget=Save_LCOETarget[CaseInvestigated]

DataStage2_Kite=[]
for i in range(len(IdxInKite)):
    site=IdxInKite[i]
    for j in range(len(IdxInKite)):
        design=IdxInKite[j]

        DataStage2_Kite.append({"X":WindKiteTrsData["X_Y_Vecs"][site,0],
                                "Y":WindKiteTrsData["X_Y_Vecs"][site,1],
                                "Desgin":design,
                                "Site":site,
                                "uopt": WindKiteTrsData["uopt_vecs"][design,:],
                                "SiteIdx_s2":i,
                                "DesignIdx_s2":j})   


In [10]:
# I have 64GB of RAM, so I can run 10 envs at the same time. 
#2500 runs = 30 minutes

def UpdateGenerationTS (Data, NumEnvs=15):
    Envs=[matlab.engine.start_matlab() for i in range(NumEnvs)] #Create envs
    for i in range(NumEnvs):
        Envs[i].cd(r'C:\Users\Remote\Desktop\Projects\OceanProject4_1_Ver\PortOpt_KiteWind_MaxGenLCOE\KiteLF_Optimization', nargout=0)
        Envs[i].eval("File='DataSetPlatform.mat';"+"load(File);" ,nargout=0)

    cmd1=[]
    cmd2=[]

    for i in range(len(Data)):
        X=Data[i]["X"]
        Y=Data[i]["Y"]
        uopt  =Data[i]["uopt"]

        cmd1_tmp = str('xSite =['+str(X)+','+str(Y)+'];')
        cmd2_tmp = str('uGeo =['+str(uopt[0])+','+str(uopt[1])+','+str(uopt[2])+','+str(uopt[3])+','+str(15000)+'];')
        cmd1.append(cmd1_tmp)
        cmd2.append(cmd2_tmp)

    G_count=0
    NumRuns=0
    while G_count!=len(Data):

        G_count=G_count+NumRuns

        print("%.2f %% complete" % (G_count/len(Data)*100))

        NumRuns=np.min([len(Envs),len(Data)-G_count]) #Number of runs to do in parallel
        
        for i in range(0,NumRuns,1):
            Envs[i].eval(cmd1[G_count+i],nargout=0) #Inputs
            Envs[i].eval(cmd2[G_count+i],nargout=0) #Inputs

        #Separate loop to run in parallel (only for running in parallel)
        for i in range(0,NumRuns,1):
            Envs[i].powerFunc_Python(nargout=0,background=True) #Solve problem in background

        for i in range(0,NumRuns,1):   
            Jopt_vec  =Envs[i].workspace["Jopt_vec"][0]
            theta_vec =Envs[i].workspace["theta_vec"][0]
            l_vec     =Envs[i].workspace["l_vec"][0]

            Data[G_count+i]["Jopt_vec"]=Jopt_vec
            Data[G_count+i]["theta_vec"]=theta_vec
            Data[G_count+i]["l_vec"]=l_vec

    for i in range(NumEnvs):
        Envs[i].quit()

    return Data

Data=UpdateGenerationTS(DataStage2_Kite)

0.00 % complete
16.53 % complete
33.06 % complete
49.59 % complete
66.12 % complete
82.64 % complete
99.17 % complete
100.00 % complete


In [22]:
for i in range(len(Data)):
    SiteIdx_s2=Data[i]["SiteIdx_s2"]
    DesignIdx_s2=Data[i]["DesignIdx_s2"]
    KiteEnergy_s2[SiteIdx_s2,DesignIdx_s2,:]=np.asarray(Data[i]["Jopt_vec"])[0]/1000 #MW

## Optimize with limited designs

In [23]:
MILP_s2 = ConcreteModel()
BM=BigM
# Create Sets
MILP_s2.SiteWind = RangeSet(0,NumWindSites_s2-1)
MILP_s2.SiteKite = RangeSet(0,NumKiteSites_s2-1)
MILP_s2.Designs  = RangeSet(0,NumKiteDesigns_s2-1)
MILP_s2.TimeSteps = RangeSet(0,NumTimeSteps-1)

# Create Variables
MILP_s2.Y_Wind = Var(MILP_s2.SiteWind, domain=NonNegativeIntegers)# Integer variable to track the number of wind turbines used per site location
MILP_s2.Y_Kite = Var(MILP_s2.SiteKite, MILP_s2.Designs, domain=NonNegativeIntegers)# Integer variable to track the number of wind turbines used per site location

MILP_s2.w = Var(MILP_s2.Designs , domain=Binary)# Binary variable to track the center of the energy collection system

MILP_s2.Delta = Var(MILP_s2.TimeSteps, domain=NonNegativeReals) #Curtailment variable

#Objective Function
def objective_rule(MILP_s2):   
    EGWind=sum(MILP_s2.Y_Wind[i]*np.average(WindEnergy_s2[i,:]) for i in MILP_s2.SiteWind) #Energy generation from wind turbines
    EGKite=sum(MILP_s2.Y_Kite[i,d]*np.average(KiteEnergy_s2[i,d,:]) for i in MILP_s2.SiteKite for d in MILP_s2.Designs) #Energy generation from kite turbines

    TotalCurtailment=sum(MILP_s2.Delta[t] for t in MILP_s2.TimeSteps)/NumTimeSteps #Average curtailment MW

    Obj=(EGWind + EGKite - TotalCurtailment)#*24*365#Mwh/year
    return Obj

MILP_s2.OBJ = Objective(rule = objective_rule, sense=maximize)

#Constraints
#Maximum number of turbines per site location wind
def MaxTurbinesCell_Wind_rule(MILP_s2,i):
    return MILP_s2.Y_Wind[i]<=MaxNumWindPerSite_s2[i]
MILP_s2.Turbines_Cell_Wind = Constraint(MILP_s2.SiteWind, rule=MaxTurbinesCell_Wind_rule)

#Maximum number of turbines per site location kite
def MaxTurbinesCell_Kite_rule(MILP_s2,i):
    return sum(MILP_s2.Y_Kite[i,d] for d in MILP_s2.Designs) <= MaxNumKitesPerSite_s2[i]
MILP_s2.Turbines_Cell_Kite = Constraint(MILP_s2.SiteKite, rule=MaxTurbinesCell_Kite_rule)

#Curtailment constraint
def Curtailment_rule(MILP_s2,t):
    EGWind=sum(MILP_s2.Y_Wind[i]*WindEnergy_s2[i,t] for i in MILP_s2.SiteWind) #Energy generation from wind turbines
    EGKite=sum(MILP_s2.Y_Kite[i,d]*np.average(KiteEnergy_s2[i,d,:]) for i in MILP_s2.SiteKite for d in MILP_s2.Designs) #Energy generation from kite turbines
    return -MILP_s2.Delta[t]+ EGWind+ EGKite<=MaxPowerTransmission
MILP_s2.Curtailment = Constraint(MILP_s2.TimeSteps, rule=Curtailment_rule)

#---Choose center collection system - End

#LCOE Target
def LCOETarget_S1(MILP_s2,LCOE_Max):  
    EG_Wind=sum(MILP_s2.Y_Wind[i]*np.average(WindEnergy[i,:]) for i in MILP_s2.SiteWind) #Average MW for wind
    EG_Kite=sum(MILP_s2.Y_Kite[i,d]*np.average(KiteEnergy_s2[i,d,:]) for i in MILP_s2.SiteKite for d in MILP_s2.Designs) #Average MW for kite
    TotalCurtailment=sum(MILP_s2.Delta[i] for i in MILP_s2.TimeSteps)/NumTimeSteps #Average curtailment MW
    
    MWh=(EG_Kite+EG_Wind-TotalCurtailment)*24*365 #MWh

    Cost_Wind=sum(MILP_s2.Y_Wind[i]*AnnualizedCostWind[i] for i in MILP_s2.SiteWind)
    Cost_Kite=sum(MILP_s2.Y_Kite[i,d]*AnnualizedCostKite[d] for i in MILP_s2.SiteKite for d in MILP_s2.Designs)
    Cost_Transmission=AnnualizedCostTransmission[TransLoc]
    Cost=Cost_Wind+Cost_Kite+Cost_Transmission

    return Cost<=LCOE_Max*MWh # 300 is a big M for the total number of turbines installed       

LCOE_Target=LCOETarget_S1(MILP_s2,LCOETarget)
MILP_s2.LCOE_Target = Constraint(rule=LCOE_Target)

def TrackDesigns(MILP_s2,d):  
    return sum(MILP_s2.Y_Kite[i,d] for i in MILP_s2.SiteKite)<=MILP_s2.w[d]*BM 
MILP_s2.TrackDesigns = Constraint(MILP_s2.Designs, rule=TrackDesigns)

def CountDesigns(MILP_s2):  
    return sum(MILP_s2.w[d] for d in MILP_s2.Designs)<=MaxNumDesigns
MILP_s2.CountDesigns = Constraint(rule=CountDesigns)

#MILP_s2.SetNumTurbines= Constraint(expr=sum(MILP_s2.Y_Kite[i,d] for i in MILP_s2.SiteKite for d in MILP_s2.Designs)==100)
MILP_s2.SetNumTurbines= Constraint(expr=sum(MILP_s2.Y_Wind[i] for i in MILP_s2.SiteWind)==200)

#---Choose center collection system - End
#
opt = SolverFactory('gurobi', solver_io="python")
opt.options['mipgap'] = 0.05
#opt.options['max_iter'] = 500
results=opt.solve(MILP_s2, tee=True)

Set parameter MIPGap to value 0.05
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 298 rows, 405 columns and 27826 nonzeros
Model fingerprint: 0x1d535054
Variable types: 87 continuous, 318 integer (11 binary)
Coefficient statistics:
  Matrix range     [3e-02, 5e+05]
  Objective range  [1e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+07]
Presolve removed 261 rows and 260 columns
Presolve time: 0.01s
Presolved: 37 rows, 145 columns, 544 nonzeros
Variable types: 13 continuous, 132 integer (11 binary)

Root relaxation: objective 5.104442e+02, 16 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  510.44418    0    1          -  510.44418      -     -    0s
H    0     0                     507.9652311

In [24]:
Optimal_Y_Kite=np.zeros((NumKiteSites_s2, NumKiteDesigns_s2))
for i in MILP_s2.SiteKite:
    for d in MILP_s2.Designs:
        Optimal_Y_Kite[i,d]=MILP_s2.Y_Kite.get_values()[i,d]


Optimal_Y_Wind =np.array([MILP_s2.Y_Wind.get_values()[j] for j in MILP_s2.SiteWind])
Curtailment=np.array([MILP_s2.Delta.get_values()[j] for j in MILP_s2.TimeSteps])
Designs=np.array([MILP_s2.w.get_values()[d] for d in MILP_s2.Designs])


In [25]:
EG_Wind=sum(Optimal_Y_Wind[i]*np.average(WindEnergy[i,:]) for i in MILP_s2.SiteWind) #Average MW for wind
EG_Kite=sum(Optimal_Y_Kite[i,d]*np.average(KiteEnergy_s2[i,d,:]) for i in MILP_s2.SiteKite for d in MILP_s2.Designs) #Average MW for kite
TotalCurtailment=sum(Curtailment[i] for i in MILP_s2.TimeSteps)/NumTimeSteps #Average curtailment MW

MW=(EG_Kite+EG_Wind-TotalCurtailment)
MWh=(EG_Kite+EG_Wind-TotalCurtailment)*24*365 #MWh
Cost_Wind=sum(Optimal_Y_Wind[i]*AnnualizedCostWind[i] for i in MILP_s2.SiteWind)
Cost_Kite=sum(Optimal_Y_Kite[i,d]*AnnualizedCostKite[d] for i in MILP_s2.SiteKite for d in MILP_s2.Designs)
Cost_Transmission=AnnualizedCostTransmission[TransLoc]
Cost=Cost_Wind+Cost_Kite+Cost_Transmission
CurrentLCOE=Cost/MWh

print("MW NL Design: %.2f,  LCOE NL Design: %.2f \nMW L Design: %.2f,  LCOE L Design: %.2f "%(SaveTotalMWAvg[CaseInvestigated],Save_LCOE_Achieved[CaseInvestigated],MW,CurrentLCOE))

MW NL Design: 495.08,  LCOE NL Design: 130.00 
MW L Design: 507.97,  LCOE L Design: 129.78 
